### 네이버 쇼핑리뷰 감성분석

In [17]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [18]:
url = "https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt"
df = pd.read_table(url, names=['score', 'review'])

In [27]:
############ 1. 데이터/텍스트 전처리를 하세요.(결측치, 중복데이터, 한글 이외의 데이터 제거등)(20)

# 평점 4, 5 점 -> 긍정(1), 나머지는 부정(0)
df.score = df.score.apply(lambda x: 1 if x >= 4 else 0 )

# 결측치
df.isna().sum().sum()

# 중복 데이터
print(df.shape)
print(df.review.nunique())
df.drop_duplicates(subset=['review'], inplace=True)
# 한글 이외의 데이터, ''를 제거
df.review = df.review.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', regex=True).str.strip()
df.review.replace('', np.nan, inplace=True)
df.isna().sum().sum()   # 발생하지 않았음

df.review

(199335, 2)
199335


0                                                   배공빠르고 굿
1                             택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2         아주좋아요 바지 정말 좋아서 개 더 구매했어요 이가격에 대박입니다  바느질이 조금 ...
3         선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다  전...
4                         민트색상 예뻐요  옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ
                                ...                        
199995                                      장마라그런가    달지않아요
199996    다이슨 케이스 구매했어요 다이슨 슈퍼소닉 드라이기 케이스 구매했어요가격 괜찮고 배송...
199997                      로드샾에서 사는것보다 세배 저렴하네요 ㅜㅜ 자주이용할께요
199998                                         넘이쁘고 쎄련되보이네요
199999     아직 사용해보지도않았고 다른 제품을 써본적이없어서 잘 모르겠지만 ㅎㅎ 배송은 빨랐습니다
Name: review, Length: 199335, dtype: object

In [20]:
############# 2. Okt를 사용하여 한글 형태소 분석을 하세요.(10)

with open('data/불용어.txt') as st:
    lines = st.readlines()
stop_words = [line.split('\t')[0] for line in lines]

from konlpy.tag import Okt
okt = Okt()

def okt_tokenizer(text):    # tokenizer함수는 ver1의 문자열 변환과정 대신해줌
    morphs = okt.morphs(text, stem=True)
    tokens = [word for word in morphs if word not in stop_words]
    return tokens

In [38]:
#### 3. CountVectorizer와 LogisticRegression을 이용하여 이진 분류를 하되, 최적의 파라메터를 도출하고 분류 정확도를 표시하세요.(20)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.review.values, df.score.values, stratify=df.score.values,
    test_size=0.2, random_state=2023
)

In [41]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(tokenizer=okt_tokenizer)
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_test_cv = cvect.transform(X_test)

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=2023, max_iter=500)

In [47]:
lr.fit(X_train_cv, y_train)

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0

In [48]:
lr.score(X_test_cv, y_test)

AttributeError: 'LogisticRegression' object has no attribute 'coef_'

In [49]:
# pipeline 학습
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
pipeline = Pipeline([('cvect', cvect),('lr', lr)])
params = {'cvect__max_df': [300, 700], 'lr__C': [1, 10]}
grid_pipe = GridSearchCV(pipeline, params, scoring='accuracy', cv=3, n_jobs=-1)

In [50]:
%time grid_pipe.fit(X_train, y_train)

PicklingError: Could not pickle the task to send it to the workers.

In [51]:
grid_pipe.best_params_

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [ ]:
grid_pipe.best_estimator_.score(X_test, y_test)